## The purpose of this notebook is to attempt to build a graph with no neighbors while still cooperating with the PyG framework.

In [26]:
import os
import types

import h5py
import numpy as np
import pandas as pd
import requests
import torch
import torch_geometric
from sklearn import neighbors


class MerfishDataset(torch_geometric.data.InMemoryDataset):
    def __init__(
        self,
        root,
        n_neighbors=3,
        train=True,
        log_transform=True,
        non_response_genes_file="/home/roko/spatial/spatial/"
        "non_response_blank_removed.txt",
    ):
        super().__init__(root)

        # non-response genes (columns) in MERFISH
        with open(non_response_genes_file, "r", encoding="utf8") as genes_file:
            self.features = [int(x) for x in genes_file.read().split(",")]
            genes_file.close()

        # response genes (columns in MERFISH)
        self.responses = list(set(range(155)) - set(self.features))

        data_list = self.construct_graphs(n_neighbors, train, log_transform)

        with h5py.File(self.merfish_hdf5, "r") as h5f:
            self.gene_names = h5f["gene_names"][:][~self.bad_genes].astype("U")

        self.data, self.slices = self.collate(data_list)

    # from https://datadryad.org/stash/dataset/doi:10.5061/dryad.8t8s248
    url = "https://datadryad.org/stash/downloads/file_stream/67671"

    behavior_types = [
        "Naive",
        "Parenting",
        "Virgin Parenting",
        "Aggression to pup",
        "Aggression to adult",
        "Mating",
    ]
    behavior_lookup = {x: i for (i, x) in enumerate(behavior_types)}
    cell_types = [
        "Ambiguous",
        "Astrocyte",
        "Endothelial 1",
        "Endothelial 2",
        "Endothelial 3",
        "Ependymal",
        "Excitatory",
        "Inhibitory",
        "Microglia",
        "OD Immature 1",
        "OD Immature 2",
        "OD Mature 1",
        "OD Mature 2",
        "OD Mature 3",
        "OD Mature 4",
        "Pericytes",
    ]
    celltype_lookup = {x: i for (i, x) in enumerate(cell_types)}

    bad_genes = np.zeros(161, dtype=bool)
    bad_genes[[12, 13, 14, 15, 16, 144]] = True

    @property
    def raw_file_names(self):
        return ["merfish.csv", "merfish.hdf5"]

    @property
    def merfish_csv(self):
        return os.path.join(self.raw_dir, "merfish.csv")

    @property
    def merfish_hdf5(self):
        return os.path.join(self.raw_dir, "merfish.hdf5")

    def download(self):
        # download csv if necessary
        if not os.path.exists(self.merfish_csv):
            with open(self.merfish_csv, "wb") as csvf:
                csvf.write(requests.get(self.url).content)

        # process csv if necessary
        dataframe = pd.read_csv(self.merfish_csv)

        with h5py.File(self.merfish_hdf5, "w") as h5f:
            # pylint: disable=no-member
            for colnm, dtype in zip(dataframe.keys()[:9], dataframe.dtypes[:9]):
                if dtype.kind == "O":
                    data = np.require(dataframe[colnm], dtype="S36")
                    h5f.create_dataset(colnm, data=data)
                else:
                    h5f.create_dataset(colnm, data=np.require(dataframe[colnm]))

            expression = np.array(dataframe[dataframe.keys()[9:]]).astype(np.float16)
            h5f.create_dataset("expression", data=expression)

            gene_names = np.array(dataframe.keys()[9:], dtype="S80")
            h5f.create_dataset("gene_names", data=gene_names)

    def construct_graph(self, data, anid, breg, n_neighbors, log_transform):
        # get subset of cells in this slice
        good = (data.anids == anid) & (data.bregs == breg)

        # figure out neighborhood structure
        locations_for_this_slice = data.locations[good]
# LINES THAT WERE CHANGED
###############################################################################################        
        if n_neighbors == 0:
            edges = np.concatenate(
                [np.c_[np.array([i]), np.array([i])] for i in range(locations_for_this_slice.shape[0])],
                axis=0
            )
            print(edges)
        
        else:
        
            nbrs = neighbors.NearestNeighbors(
                n_neighbors=n_neighbors + 1, algorithm="ball_tree"
            )
            nbrs.fit(locations_for_this_slice)
            _, kneighbors = nbrs.kneighbors(locations_for_this_slice)
            edges = np.concatenate(
                [np.c_[kneighbors[:, 0], kneighbors[:, i + 1]] for i in range(n_neighbors)],
                axis=0,
            )
        
        edges = torch.tensor(edges, dtype=torch.long).T
############################################################################################### 
        # remove gene 144.  which is bad.  for some reason.
        subexpression = data.expression[good]
        subexpression = subexpression[:, ~self.bad_genes]

        # get behavior ids
        behavior_ids = np.array([self.behavior_lookup[x] for x in data.behavior[good]])
        celltype_ids = np.array([self.celltype_lookup[x] for x in data.celltypes[good]])
        labelinfo = np.c_[behavior_ids, celltype_ids]

        # make it into a torch geometric data object, add it to the list!

        # if we want to first log transform the data, we do it here
        # make this one return statement only changing x
        predictors_x = torch.tensor(subexpression.astype(np.float32))
        if log_transform:
            predictors_x = torch.log1p(predictors_x)

        return torch_geometric.data.Data(
            x=predictors_x,
            edge_index=edges,
            pos=torch.tensor(locations_for_this_slice.astype(np.float32)),
            y=torch.tensor(labelinfo),
            bregma=breg,
        )

    def construct_graphs(self, n_neighbors, train, log_transform=True):
        # load hdf5
        with h5py.File(self.merfish_hdf5, "r") as h5f:
            # pylint: disable=no-member
            data = types.SimpleNamespace(
                anids=h5f["Animal_ID"][:],
                bregs=h5f["Bregma"][:],
                expression=h5f["expression"][:],
                locations=np.c_[h5f["Centroid_X"][:], h5f["Centroid_Y"][:]],
                behavior=h5f["Behavior"][:].astype("U"),
                celltypes=h5f["Cell_class"][:].astype("U"),
            )

        # get the (animal_id,bregma) pairs that define a unique slice
        unique_slices = np.unique(np.c_[data.anids, data.bregs], axis=0)

        # are we looking at train or test sets?
        unique_slices = unique_slices[:150] if train else unique_slices[150:]

        # store all the slices in this list...
        data_list = []
        for anid, breg in unique_slices:
            data_list.append(
                self.construct_graph(data, anid, breg, n_neighbors, log_transform)
            )

        return data_list


class FilteredMerfishDataset(MerfishDataset):
    def __init__(
        self,
        root,
        n_neighbors=3,
        train=True,
        log_transform=True,
        non_response_genes_file="/home/roko/spatial/spatial/"
        "non_response_blank_removed.txt",
        sexes=None,
        behaviors=None,
        test_animal=None,
    ):
        self.root = root
        self.sexes = sexes
        self.behaviors = behaviors
        self.test_animal = test_animal
        original_csv_file = super().merfish_csv
        new_df = pd.read_csv(original_csv_file)
        print(f"Original Data {new_df.shape}")
        if self.sexes is not None:
            new_df = new_df[new_df["Animal_sex"].isin(self.sexes)]
        if self.behaviors is not None:
            new_df = new_df[new_df["Behavior"].isin(self.behaviors)]
        if new_df.shape[0] == 0:
            raise ValueError("Dataframe has no rows. Cannot build graph.")
        new_df.to_csv(self.root + "/raw/merfish_messi.csv", index=False)
        print(f"Filtered Data {new_df.shape}")
        # print("Filtered csv file created!")
        MerfishDataset.download(self)
        super().__init__(
            root,
            n_neighbors=n_neighbors,
            train=train,
            log_transform=log_transform,
            non_response_genes_file=non_response_genes_file,
        )
        # print("Filtered hdf5 file created!")

    #     @property
    #     def raw_file_names(self):
    #         return ["merfish_messi.csv", "merfish_messi.hdf5"]

    # THIS LINE WAS EDITED TO SHOW NEW FILE
    @property
    def merfish_csv(self):
        return os.path.join(self.raw_dir, "merfish_messi.csv")

    # THIS LINE WAS EDITED TO SHOW NEW FILE
    @property
    def merfish_hdf5(self):
        return os.path.join(self.raw_dir, "merfish_messi.hdf5")

    def construct_graphs(self, n_neighbors, train, log_transform=True):
        print(self.merfish_hdf5)
        # load hdf5
        with h5py.File(self.merfish_hdf5, "r") as h5f:
            # pylint: disable=no-member
            data = types.SimpleNamespace(
                anids=h5f["Animal_ID"][:],
                bregs=h5f["Bregma"][:],
                expression=h5f["expression"][:],
                locations=np.c_[h5f["Centroid_X"][:], h5f["Centroid_Y"][:]],
                behavior=h5f["Behavior"][:].astype("U"),
                celltypes=h5f["Cell_class"][:].astype("U"),
            )

        anid_to_bregma_count = {
            1: 12,
            2: 12,
            3: 6,
            4: 5,
            5: 6,
            6: 6,
            7: 12,
            8: 6,
            9: 6,
            10: 6,
            11: 6,
            12: 4,
            13: 4,
            14: 4,
            15: 4,
            16: 4,
            17: 4,
            18: 4,
            19: 4,
            20: 4,
            21: 4,
            22: 4,
            23: 4,
            24: 4,
            25: 4,
            26: 4,
            27: 2,
            28: 4,
            29: 4,
            30: 4,
        }

        # get the (animal_id,bregma) pairs that define a unique slice
        unique_slices = np.unique(np.c_[data.anids, data.bregs], axis=0)

        # are we looking at train or test sets?

        # if we want a specific animals
        if self.test_animal is not None:
            # we need to find which of the slices
            sorted_anids = np.sort(np.unique(data.anids))
            slices_before_test_anid = 0
            for anid in sorted_anids:
                if anid != self.test_animal:
                    slices_before_test_anid += anid_to_bregma_count[anid]
                else:
                    break

            mask_train = np.ones(unique_slices.shape[0], dtype=bool)
            mask_train[
                slices_before_test_anid : (
                    slices_before_test_anid + anid_to_bregma_count[self.test_animal]
                )
            ] = 0
            unique_slices = (
                unique_slices[(1 - mask_train).astype("bool")]
                if not train
                else unique_slices[mask_train]
            )
        else:
            min_animal = anid_to_bregma_count[np.min(data.anids)]
            unique_slices = (
                unique_slices[min_animal:] if train else unique_slices[:min_animal]
            )

        # store all the slices in this list...
        data_list = []
        for anid, breg in unique_slices:
            data_list.append(
                self.construct_graph(data, anid, breg, n_neighbors, log_transform)
            )

        return data_list

In [27]:
test_trial = FilteredMerfishDataset("../data", n_neighbors=0, sexes=["Female"], behaviors=["Naive"])

Original Data (1027848, 170)
Filtered Data (205348, 170)
../data/raw/merfish_messi.hdf5
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [6088 6088]
 [6089 6089]
 [6090 6090]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [6260 6260]
 [6261 6261]
 [6262 6262]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [6325 6325]
 [6326 6326]
 [6327 6327]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [6132 6132]
 [6133 6133]
 [6134 6134]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5816 5816]
 [5817 5817]
 [5818 5818]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5690 5690]
 [5691 5691]
 [5692 5692]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5674 5674]
 [5675 5675]
 [5676 5676]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5378 5378]
 [5379 5379]
 [5380 5380]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5724 5724]
 [5725 5725]
 [5726 5726]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5281 5281]
 [5282 5282]
 [5283 5283]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5358 5358]
 [5359 5359]
 [

In [28]:
test_trial[0].edge_index

tensor([[   0,    1,    2,  ..., 6088, 6089, 6090],
        [   0,    1,    2,  ..., 6088, 6089, 6090]])

In [1]:
import pandas as pd
import json
import time

import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import random_split
from torch_geometric.data import DataLoader

# from spatial.merfish_dataset import FilteredMerfishDataset, MerfishDataset
from spatial.models.monet_ae import MonetAutoencoder2D, TrivialAutoencoder
from spatial.train import train
from spatial.predict import test

import torch

import hydra
from hydra.experimental import compose, initialize

behaviors = ["Naive"]
sexes = ["Female"]

with open('animal_id.json') as json_file:
    animals = json.load(json_file)

loss_dict = {}
time_dict = {}
loss_excitatory_dict = {}
loss_inhibitory_dict = {}

for behavior in behaviors:
    for sex in sexes:
        try:
            animal_list = animals[behavior][sex]
        except KeyError:
            continue
        behavior = [behavior]
        sex = [sex]
        # print(behavior, sex, animal_list)
        start = time.time()
        with initialize(config_path="../config"):
            cfg_from_terminal = compose(config_name="config")
            # update the behavior to get the model of interest
            OmegaConf.update(cfg_from_terminal, "datasets.dataset.behaviors", behavior)
            OmegaConf.update(cfg_from_terminal, "datasets.dataset.sexes", sex)
            OmegaConf.update(cfg_from_terminal, "datasets.dataset.test_animal", 1)
            OmegaConf.update(cfg_from_terminal, "n_neighbors", 0)
            model = train(cfg_from_terminal)
            output = test(cfg_from_terminal)
            trainer, l1_losses, inputs, gene_expressions, celltypes, test_results = output
            MAE = test_results[0]['test_loss: mae_response']
            excitatory_cells = (celltypes == 6).nonzero(as_tuple=True)[0]
            MAE_excitatory = torch.abs(torch.index_select((gene_expressions-inputs)[excitatory_cells], 1, torch.tensor(model.responses))).mean().item()
            inhibitory_cells = (celltypes == 7).nonzero(as_tuple=True)[0]
            MAE_inhibitory = torch.abs(torch.index_select((gene_expressions-inputs)[inhibitory_cells], 1, torch.tensor(model.responses))).mean().item()
        end = time.time()
#             time_dict[f"{sex}_{behavior}_{animal}"] = end-start
#             loss_dict[f"{sex}_{behavior}_{animal}"] = MAE
#             loss_excitatory_dict[f"{sex}_{behavior}_{animal}"] = MAE_excitatory
#             loss_inhibitory_dict[f"{sex}_{behavior}_{animal}"] = MAE_inhibitory

#             with open("deepST_MAE.json", "w") as outfile:
#                 json.dump(loss_dict, outfile, indent=4)

#             with open("deepST_time.json", "w") as outfile:
#                 json.dump(time_dict, outfile, indent=4)

#             with open("deepST_MAE_excitatory.json", "w") as outfile:
#                 json.dump(loss_excitatory_dict, outfile, indent=4)

#             with open("deepST_MAE_inhibitory.json", "w") as outfile:
#                 json.dump(loss_inhibitory_dict, outfile, indent=4)

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/experimental/initialize.py:35: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/experimental/compose.py:18: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'predict/default': Usage of deprecated keyword in package head

Original Data (1027848, 170)
Filtered Data (205348, 170)
/home/roko/spatial/data/raw/merfish_messi.hdf5
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [6088 6088]
 [6089 6089]
 [6090 6090]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [6260 6260]
 [6261 6261]
 [6262 6262]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [6325 6325]
 [6326 6326]
 [6327 6327]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [6132 6132]
 [6133 6133]
 [6134 6134]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5816 5816]
 [5817 5817]
 [5818 5818]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5690 5690]
 [5691 5691]
 [5692 5692]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5674 5674]
 [5675 5675]
 [5676 5676]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5378 5378]
 [5379 5379]
 [5380 5380]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5724 5724]
 [5725 5725]
 [5726 5726]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5281 5281]
 [5282 5282]
 [5283 5283]]
[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5358 5358]

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:286: LightningDeprecationWarning: Passing `Trainer(accelerator='dp')` has been deprecated in v1.5 and will be removed in v1.7. Use `Trainer(strategy='dp')` instead.
  rank_zero_deprecation(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name            | Type                    | Params
------------------------------------------------------------
0 | encoder_n

Validation sanity check: 0it [00:00, ?it/s]

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/overrides/data_parallel.py:97: UserWarning: Could not determine on which device the inputs are. When using DataParallel (strategy='dp'), be aware that in case you are using self.device in your code, it will reference only the root device.
  rank_zero_warn(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch 

...

[[   0    0]
 [   1    1]
 [   2    2]
 ...
 [5581 5581]
 [5582 5582]
 [5583 5583]]


/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/overrides/data_parallel.py:97: UserWarning: Could not determine on which device the inputs are. When using DataParallel (strategy='dp'), be aware that in case you are using self.device in your code, it will reference only the root device.
  rank_zero_warn(
TEST Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
---------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  10.487         	|  100 %          	|
---------------------------------------------------------------------------------------------------------------------------------------
run_test_evaluation                	|  10.441         	|1              	|  10.441         	|  99.56          	|
eva

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.20122438669204712,
 'test_loss: mae_response': 0.32917433977127075,
 'test_loss: mse': 0.20576705038547516}
--------------------------------------------------------------------------------
